# Dawid Skene (Pooled: Multinomial Model)
Uses a single confusion matrix instead of one per worker. No need to track individual worker behavior.

However, treats all workers equally reliable/unreliable.

Instead of indexing by worker (pi[k, j, l]), we can use the shared confusion matrix pi[j, l].
The probability of a task belonging to class j is computed using all worker responses at once. (So basically we have a single worker, right?)



## What might be worth investigating:

- `_m_step`:
    ```python
    np.where(denom <= 0, -1e9, denom)
    ```
    `-1e9` could be changed to epsilon (`np.finfo(float).eps`) or `np.maximum(denom, eps)` or something similar?

- `_e_step`:
    Maybe the e-step should rely more on log `log_T = np.log(self.rho) + (self.crowd_matrix * np.log(self.pi)).sum(axis=(1, 2))` or something similar to this.
    Can double loop (over `i` and `j`) be solved by `np.einsum`?


```python
# Summing over l in y -> shape (N, J)
sum_y_nj = np.sum(self.crowd_matrix, axis=1)  # Summing over L

# Summing over n with weighting by t -> shape (K, J)
numerator = np.dot(self.T.T, sum_y_nj)  # (K, N) @ (N, J) -> (K, J)

# Summing over i (K) in the denominator
denom = np.sum(numerator, axis=1, keepdims=True)  # (J, 1)

```
should be equivalent to:
```python

aggregated_votes = np.einsum(
    "tq, tij -> qj", self.T, self.crowd_matrix
)  # shape (n_classes, n_classes)

denom = aggregated_votes.sum(
    axis=1, keepdims=True
)
```

In [ ]:
import warnings
from os import PathLike
from sys import getsizeof
from typing import Annotated, Generator

import numpy as np
import sparse as sp
from annotated_types import Ge
from loguru import logger
from memory_profiler import profile
from pydantic import validate_call
from tqdm.auto import tqdm

from peerannot.models.template import CrowdModel,AnswersDict

# load data
from toy_data import votes, N_CLASSES, N_WORKERS

from peerannot.models.aggregation.DS import DawidSkene

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

class DawidSkeneShared(DawidSkene):
    """
    =============================
    Dawid and Skene model (1979)
    =============================

    Assumptions:
    - independent workers

    Using:
    - EM algorithm

    Estimating:
    - One confusion matrix for each workers
    """
    def _m_step(
        self,
    ) -> None:
        """Maximizing log likelihood (see eq. 2.3 and 2.4 Dawid and Skene 1979)

        Returns:
            :math:`\\rho`: :math:`(\\rho_j)_j` probabilities that instance has true response j if drawn at random (class marginals)
            pi: number of times worker k records l when j is correct
        """
        self.rho = self.T.sum(0) / self.n_task

        aggregated_votes = np.einsum(
            "tq, tij -> qj", self.T, self.crowd_matrix
        )  # shape (n_classes, n_classes)
        denom = aggregated_votes.sum(
            axis=1, keepdims=True
        )
        # self.shared_pi = aggregated_votes/ np.where(denom <=0, -1e9, denom).reshape(-1,1)
        self.shared_pi = np.where(denom > 0, aggregated_votes / denom, 0)

    def _e_step(self) -> None:
        """Estimate indicator variables using a shared confusion matrix"""

        T = np.zeros((self.n_task, self.n_classes))

        # use mask instead of power
        for i in range(self.n_task):
            for j in range(self.n_classes):
                num = (
                    np.prod(
                        np.power(
                            self.shared_pi[j, :], self.crowd_matrix[i, :, :]
                        )
                    )
                    * self.rho[j]
                )
                T[i, j] = num


        self.denom_e_step = T.sum(axis=1, keepdims=True)
        self.T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)


In [3]:
dss = DawidSkeneShared(answers=votes, n_workers=N_WORKERS, n_classes=N_CLASSES, sparse=False)
dss.run()
dss.get_answers()

2025-03-21 14:25:50.058 | DEBUG    | __main__:init_crowd_matrix:95 - Dense crowd matrix  5904
2025-03-21 14:25:50.059 | DEBUG    | __main__:init_T:103 - Size of T before calc: 1568
2025-03-21 14:25:50.060 | DEBUG    | __main__:init_T:107 - Size of T: 1568
Finished: 100%|██████████| 50/50 [00:00<00:00, 354.03it/s]       


array([2, 2, 8, 1, 4, 0, 5, 3, 7, 8, 0, 5, 8, 5, 4, 7, 4, 7, 5, 7])

In [4]:
dss.T[4]

array([1.34007145e-48, 0.00000000e+00, 0.00000000e+00, 2.22660942e-01,
       7.51323096e-01, 1.12126806e-29, 2.60159623e-02, 0.00000000e+00,
       0.00000000e+00])

In [5]:
dss.log_likelihood()

np.float64(0.042191024486190984)

In [6]:
from peerannot.models import DawidSkene
ds = DawidSkene(answers=votes, n_workers=N_WORKERS, n_classes=N_CLASSES, sparse=False)
ds.run(maxiter=1,verbose=True)
ds.get_answers()

2025-03-21 14:25:50.243 | DEBUG    | peerannot.models.aggregation.DS:init_crowd_matrix:128 - Dense crowd matrix  5904
2025-03-21 14:25:50.244 | DEBUG    | peerannot.models.aggregation.DS:__init__:93 - Dense Crowd matrix5904
2025-03-21 14:25:50.245 | DEBUG    | peerannot.models.aggregation.DS:init_T:135 - Size of T before calc: 1568
2025-03-21 14:25:50.246 | DEBUG    | peerannot.models.aggregation.DS:init_T:139 - Size of T: 1568
Finished: 100%|██████████| 1/1 [00:00<00:00, 327.60it/s]
/home/jozef/Desktop/repos/peerannot/peerannot/models/aggregation/DS.py:335: DidNotConverge: DawidSkene did not converge: err=inf, epsilon=1e-06.
  return self.run_dense(


array([2, 2, 8, 1, 4, 0, 3, 3, 7, 1, 0, 3, 8, 3, 4, 7, 4, 7, 3, 7])